<a href="https://colab.research.google.com/github/suvam001/Pyspark/blob/main/Term_deposit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Bank Marketing Campaign- Predicting Term Deposit**

#**01 Install Spark**

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

# **02 Load Data**

In [ ]:
!ls spark-3.0.1-bin-hadoop3.2


ls: cannot access 'spark-3.0.1-bin-hadoop3.2': No such file or directory


In [ ]:
data_file = "Bank_train.csv"
raw_data = sc.textFile(data_file)

print ("Train data size is ", raw_data.count())

Train data size is  41188


In [ ]:
test_data_file = "Bank_test.csv"

test_raw_data = sc.textFile(test_data_file)

print ("Test data size is ", test_raw_data.count())

Test data size is  4119


#**03 Exploratory Data Analysis -  MLlib Statistics** <br>


In [ ]:
#sc = spark.sparkContext
datafile = "Bank.csv"
raw_data = sc.textFile(datafile)

In [ ]:
raw_data.take(6)

['0.48148148148148145,0.05307035380235868,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844,housemaid,married,basic.4y,no,no,no,telephone,may,mon,nonexistent,no,cont',
 '0.4938271604938272,0.03029686864579097,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844,services,married,high.school,unknown,no,no,telephone,may,mon,nonexistent,no,cont',
 '0.24691358024691357,0.045953639690931275,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844,services,married,high.school,no,yes,no,telephone,may,mon,nonexistent,no,cont',
 '0.2839506172839506,0.030703538023586822,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844,admin.,married,basic.6y,no,no,no,telephone,may,mon,nonexistent,no,cont',
 '0.48148148148148145,0.06242374949166328,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844,services,married,high.sc

**An RDD of dense vectors**

Categorical variables are removed

In [ ]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [9,10,11,12,13,14,15,16,17,18,19,20]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data = raw_data.map(parse_interaction)
vector_data.take(2)

[array([0.48148148, 0.05307035, 0.        , 0.        , 0.9375    ,
        0.69875292, 0.60251046, 0.95737928, 0.85984848]),
 array([0.49382716, 0.03029687, 0.        , 0.        , 0.9375    ,
        0.69875292, 0.60251046, 0.95737928, 0.85984848])]

###**Summary Statistics**

In [ ]:
from pyspark.mllib.stat import Statistics 
from math import sqrt 

# Compute column summary statistics.
summary = Statistics.colStats(vector_data)



In [ ]:
print ("Campaign:")
print (" Mean: ", round(summary.mean()[2],3))
print (" St. deviation: ", round(sqrt(summary.variance()[2]),3))
print (" Max value: ", round(summary.max()[2],3))
print (" Min value: ", round(summary.min()[2],3))
print (" Total value count: ", summary.count())
print (" Number of non-zero values: ", summary.numNonzeros()[2])

Campaign:
 Mean:  0.029
 St. deviation:  0.05
 Max value:  1.0
 Min value:  0.0
 Total value count:  41188
 Number of non-zero values:  23546.0


In [ ]:
print ("Previous no of contacts:")
print (" Mean: ", round(summary.mean()[3],3))
print (" St. deviation: ", round(sqrt(summary.variance()[3]),3))
print (" Max value: ", round(summary.max()[3],3))
print (" Min value: ", round(summary.min()[3],3))
print (" Total value count: ", summary.count())
print (" Number of non-zero values: ", summary.numNonzeros()[3])

Previous no of contacts:
 Mean:  0.025
 St. deviation:  0.071
 Max value:  1.0
 Min value:  0.0
 Total value count:  41188
 Number of non-zero values:  5625.0


####**Summary Statistics by label**

In [ ]:
def parse_interaction_with_key(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [9,10,11,12,13,14,15,16,17,18,19,20]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return (line_split[19], np.array([float(x) for x in clean_line_split]))


In [ ]:
label_vector_data = raw_data.map(parse_interaction_with_key)

In [ ]:
label_vector_data.take(2)

[('no', array([0.48148148, 0.05307035, 0.        , 0.        , 0.9375    ,
         0.69875292, 0.60251046, 0.95737928, 0.85984848])),
 ('no', array([0.49382716, 0.03029687, 0.        , 0.        , 0.9375    ,
         0.69875292, 0.60251046, 0.95737928, 0.85984848]))]

In [ ]:
Yes_label_data = label_vector_data.filter(lambda x: x[0]=='yes')

In [ ]:
Yes_label_data.take(2)

[('yes', array([0.2962963 , 0.32025214, 0.        , 0.        , 0.9375    ,
         0.69875292, 0.60251046, 0.95737928, 0.85984848])),
 ('yes', array([0.39506173, 0.21187475, 0.        , 0.        , 0.9375    ,
         0.69875292, 0.60251046, 0.95737928, 0.85984848]))]

In [ ]:
Yes_summary = Statistics.colStats(Yes_label_data.values())


In [ ]:
print ("Statistics for label: ", "yes")
print (" Mean: ", round(Yes_summary.mean()[2],3))
print (" St. deviation: ", round(sqrt(Yes_summary.variance()[2]),3))
print (" Max value: ", round(Yes_summary.max()[2],3))
print (" Min value: ", round(Yes_summary.min()[2],3))
print (" Total value count: ", Yes_summary.count())
print (" Number of non-zero values: ", Yes_summary.numNonzeros()[2])

Statistics for label:  yes
 Mean:  0.019
 St. deviation:  0.03
 Max value:  0.4
 Min value:  0.0
 Total value count:  4640
 Number of non-zero values:  2340.0


####**Summary Statistics by label('0' & '1') using a function**


In [ ]:
def summary_by_label(raw_data, label):
    label_vector_data = raw_data.map(parse_interaction_with_key).filter(lambda x: x[0]==label)
    return Statistics.colStats(label_vector_data.values())

In [ ]:
Yes_sum = summary_by_label(raw_data, "yes")

print ("Campaign Statistics for label: ", "yes")
print (" Mean: ", round(Yes_sum.mean()[2],3))
print (" St. deviation: ", round(sqrt(Yes_sum.variance()[2]),3))
print (" Max value: ", round(Yes_sum.max()[2],3))
print (" Min value: ", round(Yes_sum.min()[2],3))
print (" Total value count: ", Yes_sum.count())
print (" Number of non-zero values: ", Yes_sum.numNonzeros()[2])

Campaign Statistics for label:  yes
 Mean:  0.019
 St. deviation:  0.03
 Max value:  0.4
 Min value:  0.0
 Total value count:  4640
 Number of non-zero values:  2340.0


In [ ]:
No_sum = summary_by_label(raw_data, "no")

print ("Campaign Statistics for label: ", "No")
print (" Mean: ", round(No_sum.mean()[2],3))
print (" St. deviation: ", round(sqrt(No_sum.variance()[2]),3))
print (" Max value: ", round(No_sum.max()[2],3))
print (" Min value: ", round(No_sum.min()[2],3))
print (" Total value count: ", No_sum.count())
print (" Number of non-zero values: ", No_sum.numNonzeros()[2])

Campaign Statistics for label:  No
 Mean:  0.03
 St. deviation:  0.052
 Max value:  1.0
 Min value:  0.0
 Total value count:  36548
 Number of non-zero values:  21206.0


In [ ]:
label_list = ["yes","no"]

In [ ]:
from time import time

In [ ]:
t0 = time()
stats_by_label = [(label, summary_by_label(raw_data, label)) for label in label_list]
tt = time()-t0
print('time taken ',tt, 'seconds')
# with CPU time taken  225.09664249420166 seconds
# with GPU time taken  194.15603494644165 seconds

time taken  1.5835752487182617 seconds


In [ ]:
stats_by_label

[('yes',
  <pyspark.mllib.stat._statistics.MultivariateStatisticalSummary at 0x7f3300c3d550>),
 ('no',
  <pyspark.mllib.stat._statistics.MultivariateStatisticalSummary at 0x7f3300c3d6d0>)]

In [ ]:
duration_by_label2 = [(stat[0],float(stat[1].mean()[2]), float(sqrt(stat[1].variance()[2])), float(stat[1].min()[2]), float(stat[1].max()[2]), int(stat[1].count())) for stat in stats_by_label]

In [ ]:
duration_by_label2

[('yes',
  0.0191222570532915,
  0.030295357336210065,
  0.0,
  0.39999999999999997,
  4640),
 ('no',
  0.029692459231695282,
  0.05224433022838142,
  0.0,
  0.9999999999999999,
  36548)]

In [ ]:
duration_by_label2A = [(stat[0],float(stat[1].mean()[3]), float(sqrt(stat[1].variance()[3])), float(stat[1].min()[3]), float(stat[1].max()[3]), int(stat[1].count())) for stat in stats_by_label]

In [ ]:
duration_by_label2A

[('yes',
  0.07038177339901479,
  0.12290630557236663,
  0.0,
  0.8571428571428571,
  4640),
 ('no', 0.018910552072421385, 0.05845702815811167, 0.0, 1.0, 36548)]

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

#stats_by_label_df = pd.DataFrame.from_items(duration_by_label, columns=["Mean", "Std Dev", "Min", "Max", "Count"], orient='index')
stats_by_label_df2 = pd.DataFrame(duration_by_label2, columns=[' ',"Mean", "Std Dev", "Min", "Max", "Count"])

####**Summary Statistics for all the variables by label ('yes' & 'no') and putting it into a dataframe**

In [ ]:
print ("Campaign statistics, by label")
stats_by_label_df2

Campaign statistics, by label


,,Mean,Std Dev,Min,Max,Count
0,yes,0.019122,0.030295,0.0,0.4,4640
1,no,0.029692,0.052244,0.0,1.0,36548


In [ ]:
def get_variable_stats_df2(stats_by_label, column_i):
    column_stats_by_label = [
        (stat[0], float(stat[1].mean()[column_i]), float(sqrt(stat[1].variance()[column_i])), float(stat[1].min()[column_i]), float(stat[1].max()[column_i]), int(stat[1].count())) 
        for stat in stats_by_label
    ]
    return pd.DataFrame(column_stats_by_label, columns=[' ',"Mean", "Std Dev", "Min", "Max", "Count"])

In [ ]:
print ("Duration statistics, by label")
get_variable_stats_df2(stats_by_label,2)

Duration statistics, by label


,,Mean,Std Dev,Min,Max,Count
0,yes,0.019122,0.030295,0.0,0.4,4640
1,no,0.029692,0.052244,0.0,1.0,36548


In [ ]:
print ("Previous no of contacts statistics, by label")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,3)

Previous no of contacts statistics, by label


,,Mean,Std Dev,Min,Max,Count
0,yes,0.070382,0.122906,0.0,0.857143,4640
1,no,0.018911,0.058457,0.0,1.000000,36548


In [ ]:
print ("Emp.var.rate statistics, by label")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,4)

Emp.var.rate statistics, by label


,,Mean,Std Dev,Min,Max,Count
0,yes,0.451365,0.338255,0.0,1.0,4640
1,no,0.760182,0.308944,0.0,1.0,36548


In [ ]:
print ("Age statistics")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,0)

Age statistics


,,Mean,Std Dev,Min,Max,Count
0,yes,0.295224,0.170833,0.0,1.000000,4640
1,no,0.282854,0.122199,0.0,0.962963,36548


In [ ]:
print ("Duration statistics")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,1)

Duration statistics


,,Mean,Std Dev,Min,Max,Count
0,yes,0.112483,0.081572,0.007523,0.853802,4640
1,no,0.044905,0.042110,0.000000,1.000000,36548


In [ ]:
print ("Cons.price.idx statistics")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,5)

Cons.price.idx statistics


,,Mean,Std Dev,Min,Max,Count
0,yes,0.449488,0.263696,0.0,1.0,4640
1,no,0.546671,0.217846,0.0,1.0,36548


In [ ]:
print ("cons.conf.idx statistics")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,6)

cons.conf.idx statistics


,,Mean,Std Dev,Min,Max,Count
0,yes,0.460678,0.25689,0.0,1.0,4640
1,no,0.427067,0.18373,0.0,1.0,36548


In [ ]:
print ("euribor3m statistics")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,7)

euribor3m statistics


,,Mean,Std Dev,Min,Max,Count
0,yes,0.337596,0.395057,0.0,1.0,4640
1,no,0.720356,0.371387,0.0,1.0,36548


In [ ]:
print ("nr.employed statistics")
#get_variable_stats_df(stats_by_label,1)
get_variable_stats_df2(stats_by_label,8)

nr.employed statistics


,,Mean,Std Dev,Min,Max,Count
0,yes,0.496974,0.331082,0.0,1.0,4640
1,no,0.803548,0.244410,0.0,1.0,36548


###**Correlations**
Spearman’s correlation method is used here to  calculate the  pairwise correlation between the variables

In [ ]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print ("Sample size is ",sample_size , "of ", total_size)

Sample size is  3951 of  41188


In [ ]:
import numpy as np

def parse_interaction(line):
    line_split = line.split(",")
    # keep just numeric and logical values
    symbolic_indexes = [9,10,11,12,13,14,15,16,17,18,19,20]
    clean_line_split = [item for i,item in enumerate(line_split) if i not in symbolic_indexes]
    return np.array([float(x) for x in clean_line_split])

vector_data_sample = raw_data_sample.map(parse_interaction)
vector_data_sample.take(2)

[array([0.48148148, 0.05307035, 0.        , 0.        , 0.9375    ,
        0.69875292, 0.60251046, 0.95737928, 0.85984848]),
 array([0.08641975, 0.07726718, 0.        , 0.        , 0.9375    ,
        0.69875292, 0.60251046, 0.95737928, 0.85984848])]

In [ ]:
from pyspark.mllib.stat import Statistics 
correlation_matrix = Statistics.corr(vector_data_sample, method="spearman") # we have used a smaller sample here


**We have used a Pandas DataFrame here to render the correlation matrix in a more comprehensive way.**

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 50)

col_names = ["Age","Duration","Campaign","Previous","Emp.var.rate",
             "cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]


corr_df = pd.DataFrame(correlation_matrix, index=col_names, columns=col_names)

corr_df

,Age,Duration,Campaign,Previous,Emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
Age,1.000000,-0.013576,0.000105,0.011658,0.045326,0.061065,0.109334,0.046645,0.042812
Duration,-0.013576,1.000000,-0.040788,0.022859,-0.042230,0.018173,0.021399,-0.050259,-0.072333
Campaign,0.000105,-0.040788,1.000000,-0.067053,0.151612,0.085293,-0.021448,0.140353,0.142309
Previous,0.011658,0.022859,-0.067053,1.000000,-0.433654,-0.291736,-0.110958,-0.451582,-0.438258
Emp.var.rate,0.045326,-0.042230,0.151612,-0.433654,1.000000,0.669521,0.224463,0.939491,0.947609
cons.price.idx,0.061065,0.018173,0.085293,-0.291736,0.669521,1.000000,0.260643,0.499292,0.477857
cons.conf.idx,0.109334,0.021399,-0.021448,-0.110958,0.224463,0.260643,1.000000,0.238691,0.132498
euribor3m,0.046645,-0.050259,0.140353,-0.451582,0.939491,0.499292,0.238691,1.000000,0.928453
nr.employed,0.042812,-0.072333,0.142309,-0.438258,0.947609,0.477857,0.132498,0.928453,1.000000


In [ ]:
# get a boolean dataframe where true means that a pair of variables is highly correlated
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)
highly_correlated_df

,Age,Duration,Campaign,Previous,Emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
Age,False,False,False,False,False,False,False,False,False
Duration,False,False,False,False,False,False,False,False,False
Campaign,False,False,False,False,False,False,False,False,False
Previous,False,False,False,False,False,False,False,False,False
Emp.var.rate,False,False,False,False,False,False,False,True,True
cons.price.idx,False,False,False,False,False,False,False,False,False
cons.conf.idx,False,False,False,False,False,False,False,False,False
euribor3m,False,False,False,False,True,False,False,False,True
nr.employed,False,False,False,False,True,False,False,True,False


In [ ]:
# get the names of the variables so we can use them to slice the dataframe
correlated_vars_index = (highly_correlated_df==True).any()
correlated_vars_index

Age               False
Duration          False
Campaign          False
Previous          False
Emp.var.rate       True
cons.price.idx    False
cons.conf.idx     False
euribor3m          True
nr.employed        True
dtype: bool

In [ ]:
# columns that have some high correlation
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
correlated_var_names

Index(['Emp.var.rate', 'euribor3m', 'nr.employed'], dtype='object')

In [ ]:
# columns that have no high correlation
not_correlated_var_names = correlated_vars_index[correlated_vars_index!=True].index
not_correlated_var_names

Index(['Age', 'Duration', 'Campaign', 'Previous', 'cons.price.idx',
       'cons.conf.idx'],
      dtype='object')

In [ ]:
# slice it
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

,Emp.var.rate,euribor3m,nr.employed
Emp.var.rate,False,True,True
euribor3m,True,False,True
nr.employed,True,True,False


In [ ]:
#Above four steps done in one cell
# get a boolean dataframe where true means that a pair of variables is highly correlated
highly_correlated_df = (abs(corr_df) > .8) & (corr_df < 1.0)
# get the names of the variables so we can use them to slice the dataframe
correlated_vars_index = (highly_correlated_df==True).any()
correlated_var_names = correlated_vars_index[correlated_vars_index==True].index
# slice it
highly_correlated_df.loc[correlated_var_names,correlated_var_names]

,Emp.var.rate,euribor3m,nr.employed
Emp.var.rate,False,True,True
euribor3m,True,False,True
nr.employed,True,True,False


**We found the variables Emp.var.rate, euribor3m, nr.employed as highly correlated**

#**04 ML Model :  MLlib Logistic Regression**<br>


In [ ]:
data_file = "Bank_train.csv"
raw_data = sc.textFile(data_file)

print ("Train data size is ", raw_data.count())

Train data size is  41188


In [ ]:
test_data_file = "Bank_test.csv"
test_raw_data = sc.textFile(test_data_file)

print ("Test data size is ", test_raw_data.count())

Test data size is  4119


###**Creating Labelled Points**

In [ ]:
#what is LabeledPoint
# http://spark.apache.org/docs/2.2.0/mllib-data-types.html
#
from pyspark.mllib.regression import LabeledPoint
from numpy import array

def parse_interaction(line):
    line_split = line.split(",")
    # leave_out = nr.employed [9,10,11,12,13,14,15,16,17,18,19,20]
    clean_line_split = line_split[0:9]
    attack = 1.0
    if line_split[19]=='no':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [ ]:
training_data = raw_data.map(parse_interaction)
training_data.take(5)

[LabeledPoint(0.0, [0.48148148148148145,0.05307035380235868,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844]),
 LabeledPoint(0.0, [0.4938271604938272,0.03029686864579097,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844]),
 LabeledPoint(0.0, [0.24691358024691357,0.045953639690931275,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844]),
 LabeledPoint(0.0, [0.2839506172839506,0.030703538023586822,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844]),
 LabeledPoint(0.0, [0.48148148148148145,0.06242374949166328,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844])]

In [ ]:
test_data = test_raw_data.map(parse_interaction)
test_data.take(5)

[LabeledPoint(0.0, [0.16049382716049382,0.09902399349328996,0.01818181818181818,0.0,0.33333333333333337,0.269680436476996,0.19246861924686165,0.1539333484470642,0.5117424242424242]),
 LabeledPoint(0.0, [0.2716049382716049,0.0703538023586824,0.05454545454545454,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9569258671503063,0.8598484848484844]),
 LabeledPoint(0.0, [0.09876543209876543,0.0461569743798292,0.0,0.0,1.0,0.8823070927513612,0.37656903765690375,0.9811834051235548,1.0003787878787875]),
 LabeledPoint(0.0, [0.25925925925925924,0.0034566897112647418,0.03636363636363636,0.0,1.0,0.8823070927513612,0.37656903765690375,0.9805032872364543,1.0003787878787875]),
 LabeledPoint(0.0, [0.3703703703703703,0.011793411956079707,0.0,0.0,0.6875,0.3893219017926697,0.36820083682008353,0.8063931081387441,0.8780303030303038])]

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from time import time

# Build the model
t0 = time()
logit_model = LogisticRegressionWithLBFGS.train(training_data)
#logit_model = LogisticRegressionWithLBFGS.train(raw_data)
tt = time() - t0

print ("Classifier trained in ",round(tt,3), " seconds")

Classifier trained in  5.658  seconds


In [ ]:
labels_and_preds = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))

In [ ]:
labels_and_preds.take(10)

[(0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0)]

In [ ]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda x : x[0] == x[1]).count() / float(test_data.count())
tt = time() - t0

print ("Prediction made in ",round(tt,3)," seconds. Test accuracy is ",round(test_accuracy,4))

Prediction made in  0.619  seconds. Test accuracy is  0.9092


####**Test Accuracy attained is 90.92%**

In [ ]:
def parse_interaction_corr(line):
    line_split = line.split(",")
    # leave_out = 
    clean_line_split = line_split[0:4]+line_split[4:7]
    attack = 1.0
    if line_split[19]=='no':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))



###**Correlated variables are removed and evaluated the new accuracy**

In [ ]:
corr_reduced_training_data = raw_data.map(parse_interaction_corr)
corr_reduced_test_data = test_raw_data.map(parse_interaction_corr)

In [ ]:
# Build the model
t0 = time()
logit_model_2 = LogisticRegressionWithLBFGS.train(corr_reduced_training_data)
tt = time() - t0

print ("Classifier trained in ",round(tt,3)," seconds")

Classifier trained in  3.564  seconds


In [ ]:
t0 = time()
labels_and_preds = corr_reduced_test_data.map(lambda p: (p.label, logit_model_2.predict(p.features)))
tt = time() - t0
test_accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(corr_reduced_test_data.count())


print ("Prediction made in ",round(tt,3),"seconds. Test accuracy is ",round(test_accuracy,4))

Prediction made in  0.0 seconds. Test accuracy is  0.8975


####**The new Accuracy attained is 89.975%**
Here, we have reduced accuracy and also training time.  We have lost quite a lot of accuracy and have not gained a lot of execution time during training. Also prediction time didn't improve. So it is not a good step to remove correlated variables

In [ ]:
feature_names = ["Age","Duration","Campaign","Previous","Emp.var.rate",
             "cons.price.idx","cons.conf.idx","euribor3m","nr.employed"]

###**Using hypothesis testing**

Performing Persons's chi-squared (χ2) test


In [ ]:
def parse_interaction_categorical(line):
    line_split = line.split(",")
    clean_line_split = line_split[0:9]
    attack = 1.0
    if line_split[19]=='no':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [ ]:
training_data_categorical = raw_data.map(parse_interaction_categorical)

In [ ]:
training_data_categorical.take(3)

[LabeledPoint(0.0, [0.48148148148148145,0.05307035380235868,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844]),
 LabeledPoint(0.0, [0.4938271604938272,0.03029686864579097,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844]),
 LabeledPoint(0.0, [0.24691358024691357,0.045953639690931275,0.0,0.0,0.9375,0.6987529228370946,0.6025104602510458,0.9573792790750397,0.8598484848484844])]

In [ ]:
from pyspark.mllib.stat import Statistics

chi = Statistics.chiSqTest(training_data_categorical)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 30)

records = [(result.statistic, result.pValue) for result in chi]

chi_df = pd.DataFrame(data=records, index= feature_names, columns=["Statistic","p-value"])

chi_df

,Statistic,p-value
Age,1657.478261,0.0
Duration,10460.048065,0.0
Campaign,218.863377,0.0
Previous,2299.436360,0.0
Emp.var.rate,5806.707270,0.0
cons.price.idx,7665.734777,0.0
cons.conf.idx,7665.734777,0.0
euribor3m,9026.396751,0.0
nr.employed,6975.057757,0.0


#####**Chi square test**
From the dataframe obtained after Chi square test, it is clear that all the variables are signficant and none of them should be removed

In [ ]:
def parse_interaction_chi(line):
    line_split = line.split(",")
    # leave_out = [1,2,3,6,19,41]
    clean_line_split = line_split[0:9] 
    attack = 1.0
    if line_split[19]=='no':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))




In [ ]:
training_data_chi = raw_data.map(parse_interaction_chi)
test_data_chi = test_raw_data.map(parse_interaction_chi)

In [ ]:
# Build the model
t0 = time()
logit_model_chi = LogisticRegressionWithLBFGS.train(training_data_chi)
tt = time() - t0

print ("Classifier trained in ",round(tt,3)," seconds")

Classifier trained in  4.012  seconds


In [ ]:
t0 = time()
labels_and_preds = test_data_chi.map(lambda p: (p.label, logit_model_chi.predict(p.features)))
tt = time() - t0
test_accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(test_data_chi.count())

print ("Prediction made in ",round(tt,3)," seconds. Test accuracy is ", round(test_accuracy,4))

Prediction made in  0.0  seconds. Test accuracy is  0.9092


####**Hence the Test accuracy remains the same as initial that is 90.92%**